# SQL Generation with Transformer API

In [4]:
!pip uninstall torch

Looking in indexes: https://download.pytorch.org/whl/cu126
   ---------------------------------------- 0.0/6.3 MB ? eta -:--:--
   ---------------------------------------- 6.3/6.3 MB 97.9 MB/s eta 0:00:00
   ---------------------------------------- 0.0/2.7 GB ? eta -:--:--
   ---------------------------------------- 0.0/2.7 GB 121.4 MB/s eta 0:00:23
    --------------------------------------- 0.0/2.7 GB 119.4 MB/s eta 0:00:23
   - -------------------------------------- 0.1/2.7 GB 119.2 MB/s eta 0:00:23
   - -------------------------------------- 0.1/2.7 GB 118.7 MB/s eta 0:00:23
   - -------------------------------------- 0.1/2.7 GB 118.5 MB/s eta 0:00:22
   -- ------------------------------------- 0.1/2.7 GB 118.5 MB/s eta 0:00:22
   -- ------------------------------------- 0.2/2.7 GB 118.2 MB/s eta 0:00:22
   -- ------------------------------------- 0.2/2.7 GB 118.3 MB/s eta 0:00:22
   --- ------------------------------------ 0.2/2.7 GB 118.1 MB/s eta 0:00:22
   --- -----------------

In [5]:
!pip3 install torch torchvision torchaudio --index-url https://download.pytorch.org/whl/cu126

Looking in indexes: https://download.pytorch.org/whl/cu126


In [2]:
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM

In [3]:
torch.cuda.is_available()

True

In [6]:
available_memory = torch.cuda.get_device_properties(0).total_memory

In [5]:
print(available_memory)

12878086144


##Download the Model
Use any model on Colab (or any system with >30GB VRAM on your own machine) to load this in f16. If unavailable, use a GPU with minimum 8GB VRAM to load this in 8bit, or with minimum 5GB of VRAM to load in 4bit.

This step can take around 5 minutes the first time. So please be patient :)

In [8]:
model_name = "defog/sqlcoder-7b-2"
tokenizer = AutoTokenizer.from_pretrained(model_name)
if available_memory > 15e9:
    # if you have atleast 15GB of GPU memory, run load the model in float16
    model = AutoModelForCausalLM.from_pretrained(
        model_name,
        trust_remote_code=True,
        torch_dtype=torch.float16,
        device_map="auto",
        use_cache=True,
    )
else:
    # else, load in 8 bits – this is a bit slower
    model = AutoModelForCausalLM.from_pretrained(
        model_name,
        trust_remote_code=True,
        # torch_dtype=torch.float16,
        load_in_8bit=True,
        device_map="auto",
        use_cache=True,
    )

tokenizer_config.json:   0%|          | 0.00/1.84k [00:00<?, ?B/s]

c:\Users\Zahinos\anaconda3\Lib\site-packages\huggingface_hub\file_download.py:143: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\Zahinos\.cache\huggingface\hub\models--defog--sqlcoder-7b-2. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to activate developer mode, see this article: https://docs.microsoft.com/en-us/windows/apps/get-started/enable-your-device-for-development
  warnings.warn(message)


tokenizer.model:   0%|          | 0.00/500k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.84M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/515 [00:00<?, ?B/s]

...

model-00001-of-00003.safetensors:   0%|          | 0.00/4.94G [00:00<?, ?B/s]

model-00003-of-00003.safetensors:   0%|          | 0.00/3.59G [00:00<?, ?B/s]

model-00002-of-00003.safetensors:   0%|          | 0.00/4.95G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/3 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/111 [00:00<?, ?B/s]

##Set the Question & Prompt and Tokenize
Feel free to change the schema in the prompt below to your own schema

In [11]:
prompt = """### Task
Generate a SQL query to answer [QUESTION]{question}[/QUESTION]

### Instructions
- If you cannot answer the question with the available database schema, return 'I do not know'
- Remember that revenue is price multiplied by quantity
- Remember that cost is supply_price multiplied by quantity

### Database Schema
This query will run on a database whose schema is represented in this string:
CREATE TABLE products (
  product_id INTEGER PRIMARY KEY, -- Unique ID for each product
  name VARCHAR(50), -- Name of the product
  price DECIMAL(10,2), -- Price of each unit of the product
  quantity INTEGER  -- Current quantity in stock
);

CREATE TABLE customers (
   customer_id INTEGER PRIMARY KEY, -- Unique ID for each customer
   name VARCHAR(50), -- Name of the customer
   address VARCHAR(100) -- Mailing address of the customer
);

CREATE TABLE salespeople (
  salesperson_id INTEGER PRIMARY KEY, -- Unique ID for each salesperson
  name VARCHAR(50), -- Name of the salesperson
  region VARCHAR(50) -- Geographic sales region
);

CREATE TABLE sales (
  sale_id INTEGER PRIMARY KEY, -- Unique ID for each sale
  product_id INTEGER, -- ID of product sold
  customer_id INTEGER,  -- ID of customer who made purchase
  salesperson_id INTEGER, -- ID of salesperson who made the sale
  sale_date DATE, -- Date the sale occurred
  quantity INTEGER -- Quantity of product sold
);

CREATE TABLE product_suppliers (
  supplier_id INTEGER PRIMARY KEY, -- Unique ID for each supplier
  product_id INTEGER, -- Product ID supplied
  supply_price DECIMAL(10,2) -- Unit price charged by supplier
);

-- sales.product_id can be joined with products.product_id
-- sales.customer_id can be joined with customers.customer_id
-- sales.salesperson_id can be joined with salespeople.salesperson_id
-- product_suppliers.product_id can be joined with products.product_id

### Answer
Given the database schema, here is the SQL query that answers [QUESTION]{question}[/QUESTION]
[SQL]
"""

##Generate the SQL
This can be excruciatingly slow on a T4 in Colab, and can take 10-20 seconds per query. On faster GPUs, this will take ~1-2 seconds

Ideally, you should use `num_beams`=4 for best results. But because of memory constraints, we will stick to just 1 for now.

In [12]:
import sqlparse

def generate_query(question):
    updated_prompt = prompt.format(question=question)
    inputs = tokenizer(updated_prompt, return_tensors="pt").to("cuda")
    generated_ids = model.generate(
        **inputs,
        num_return_sequences=1,
        eos_token_id=tokenizer.eos_token_id,
        pad_token_id=tokenizer.eos_token_id,
        max_new_tokens=400,
        do_sample=False,
        num_beams=1,
    )
    outputs = tokenizer.batch_decode(generated_ids, skip_special_tokens=True)

    torch.cuda.empty_cache()
    torch.cuda.synchronize()
    # empty cache so that you do generate more results w/o memory crashing
    # particularly important on Colab – memory management is much more straightforward
    # when running on an inference service
    return sqlparse.format(outputs[0].split("[SQL]")[-1], reindent=True)

In [13]:
question = "What was our revenue by product in the New York region last month?"
generated_sql = generate_query(question)

In [14]:
print(generated_sql)


SELECT p.product_id,
       SUM(s.quantity * p.price) AS revenue
FROM sales s
JOIN salespeople sp ON s.salesperson_id = sp.salesperson_id
JOIN products p ON s.product_id = p.product_id
WHERE sp.region = 'New York'
  AND s.sale_date >= (CURRENT_DATE - INTERVAL '1 month')
GROUP BY p.product_id;


# Exercise
 - Complete the prompts similar to what we did in class. 
     - Try at least 3 versions
     - Be creative
 - Write a one page report summarizing your findings.
     - Were there variations that didn't work well? i.e., where GPT either hallucinated or wrong
 - What did you learn?

In [21]:
prompt = """### Task
Generate a SQL query to answer [QUESTION]{question}[/QUESTION]

### Instructions
- If you cannot answer the question with the available database schema, return 'I do not know'
- Remember that **revenue** is calculated as `products.price * sales.quantity`
- Remember that **cost** is calculated as `product_suppliers.supply_price * sales.quantity`
- Use meaningful aliases and ensure column names are fully qualified

### Database Schema
[... schema igual que el original ...]

### Answer
Given the database schema, here is the SQL query that answers [QUESTION]{question}[/QUESTION]
[SQL]
"""


In [22]:
generated_sql = generate_query(question)
print(generated_sql)


SELECT p.name AS product_name,
       SUM(s.quantity * p.price) AS total_revenue
FROM sales s
JOIN products p ON s.product_id = p.id
JOIN product_suppliers ps ON p.id = ps.product_id
WHERE ps.supplier_id IN
    (SELECT supplier_id
     FROM suppliers
     WHERE region = 'New York')
  AND s.sale_date >= (CURRENT_DATE - INTERVAL '1 month')
GROUP BY p.name
ORDER BY total_revenue DESC NULLS LAST;


In [19]:
prompt = """### Task
Generate a readable SQL query to answer [QUESTION]{question}[/QUESTION]

### Instructions
- Format the result with clear column aliases (e.g., `AS revenue`, `AS product_name`)
- Prefer `JOIN` syntax over implicit joins
- Use `DATE_TRUNC` or `EXTRACT` where appropriate for filtering by month

### Database Schema
[... schema igual que el original ...]

### Answer
The query is:
[SQL]
"""


In [20]:
generated_sql = generate_query(question)
print(generated_sql)


SELECT p.product_name,
       SUM(s.quantity * s.price) AS revenue
FROM sales s
JOIN products p ON s.product_id = p.product_id
WHERE EXTRACT(MONTH
              FROM s.sale_date) = EXTRACT(MONTH
                                          FROM CURRENT_DATE - INTERVAL '1 month')
  AND EXTRACT(YEAR
              FROM s.sale_date) = EXTRACT(YEAR
                                          FROM CURRENT_DATE - INTERVAL '1 month')
  AND p.region = 'New York'
GROUP BY p.product_name;


In [17]:
prompt = """### Task
Write a SQL query to answer [QUESTION]{question}[/QUESTION]

### Instructions
- Use **subqueries** to compute intermediate results (e.g., revenue per product)
- Avoid using `JOIN` with more than 2 tables at once. Nest queries where needed.
- Return only relevant columns in the final SELECT

### Database Schema
[... schema igual que el original ...]

### Answer
Here is the SQL query:
[SQL]
"""


In [18]:
generated_sql = generate_query(question)
print(generated_sql)


SELECT p.name,
       SUM(s.revenue) AS total_revenue
FROM Sales s
JOIN Products p ON s.product_id = p.id
WHERE s.sale_date >= (CURRENT_DATE - INTERVAL '1 month')
  AND s.sale_date < CURRENT_DATE
GROUP BY p.name
ORDER BY total_revenue DESC NULLS LAST;
